<a href="https://colab.research.google.com/github/beyza720/nlphw2/blob/main/power_multilanguage_llm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
from sklearn.model_selection import train_test_split
from sklearn.utils import resample


drive.mount('/content/drive')

import pandas as pd

train_path = '/content/drive/My Drive/Colab Notebooks/power-lv-train.tsv'
test_path = '/content/drive/My Drive/Colab Notebooks/power-lv-test.tsv'

train_power_data = pd.read_csv(train_path, sep='\t')
test_power_data = pd.read_csv(test_path, sep='\t')


# Class imbalance for power
class_0 = train_power_data[train_power_data['label'] == 0]
class_1 = train_power_data[train_power_data['label'] == 1]

class_1_oversampled = resample(class_1, replace=True, n_samples=len(class_0), random_state=42)

balanced_train_power_data = pd.concat([class_1_oversampled, class_0])


# split the train data into 0.9 train and 0.1 validation
train_power, val_power = train_test_split(
    balanced_train_power_data,
    test_size=0.1,
    stratify=balanced_train_power_data['label'],
    random_state=42
)

# print("Eğitim Veri Sayısı:", len(balanced_train_power_data))

# print("Orientation Eğitim Seti:", len(train_power))
# print("Orientation Validation Seti:", len(val_power))
# print("Orientation test seti:", len(test_power_data))

Mounted at /content/drive


In [3]:
!pip install transformers datasets torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 32.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [4]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("dbmdz/bert-base-turkish-cased")
model = AutoModelForSequenceClassification.from_pretrained(
    "dbmdz/bert-base-turkish-cased", num_labels=2
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/60.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/251k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dbmdz/bert-base-turkish-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
from datasets import Dataset
train_dataset = Dataset.from_pandas(train_power)
val_dataset = Dataset.from_pandas(val_power)
test_dataset = Dataset.from_pandas(test_power_data)

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_val_dataset = val_dataset.map(tokenize_function, batched=True)
tokenized_test_dataset = test_dataset.map(tokenize_function, batched=True)

# print(tokenized_train_dataset[0])
# print(tokenized_train_dataset[1])

# from collections import Counter
# print(Counter(train_dataset["label"]))
# print(Counter(val_dataset["label"]))

Map:   0%|          | 0/1699 [00:00<?, ? examples/s]

Map:   0%|          | 0/189 [00:00<?, ? examples/s]

Map:   0%|          | 0/1990 [00:00<?, ? examples/s]

In [6]:
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir="test_trainer")

In [7]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    report_to="none",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    logging_dir="./logs",
    logging_steps=107,
    save_steps=107,
    save_total_limit=3
)


In [8]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 8.5 MB/s eta 0:00:00


In [9]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

In [10]:
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='weighted')
    acc = accuracy_score(labels, predictions)
    return {
        'accuracy': acc,
        'precision': precision,
        'recall': recall,
        'f1': f1,
    }


In [11]:
from transformers import Trainer

In [12]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,
    compute_metrics=compute_metrics,
)

In [13]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.695500,0.658144,0.608466,0.619540,0.608466,0.600007
2,0.641800,0.584651,0.719577,0.720679,0.719577,0.719136
3,0.536400,0.540249,0.746032,0.762678,0.746032,0.741690


TrainOutput(global_step=321, training_loss=0.6245949052947333, metrics={'train_runtime': 115.5771, 'train_samples_per_second': 44.1, 'train_steps_per_second': 2.777, 'total_flos': 1341077049169920.0, 'train_loss': 0.6245949052947333, 'epoch': 3.0})